###  importing relevant libraries

In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix


### Discovering Labels

In [24]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [25]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [26]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [27]:
renamer={}

In [28]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [29]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [30]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [31]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        repetition=25
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if False:
            cm=cm//repetition
            graph_name=output_csv+"_confusion matrix.pdf"   
            plt.figure(figsize = (40,28))
            sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

In [32]:
ml_list={"DT" :DecisionTreeClassifier()}


In [33]:
find_the_way("./","clean")

[]

In [34]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
# In[48]:

In [35]:
fdict={2: ['ACK Flag Cnt',
  'Active Max',
  'Active Mean',
  'Active Min',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd IAT Mean',
  'Bwd IAT Min',
  'Bwd IAT Std',
  'Bwd IAT Tot',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Bwd Pkts/s',
  'Bwd Seg Size Avg',
  'Dst Port',
  'FIN Flag Cnt',
  'Flow Byts/s',
  'Flow Duration',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Min',
  'Flow IAT Std',
  'Flow Pkts/s',
  'Fwd Act Data Pkts',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Std',
  'Fwd Pkts/s',
  'Fwd Seg Size Avg',
  'Idle Max',
  'Idle Mean',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Min',
  'Pkt Len Std',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'Subflow Bwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 3: ['ACK Flag Cnt',
  'Active Max',
  'Active Mean',
  'Active Min',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd IAT Mean',
  'Bwd IAT Min',
  'Bwd IAT Std',
  'Bwd IAT Tot',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Bwd Pkts/s',
  'FIN Flag Cnt',
  'Flow Byts/s',
  'Flow Duration',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Min',
  'Flow IAT Std',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Min',
  'Fwd Pkt Len Std',
  'Fwd Pkts/s',
  'Idle Max',
  'Idle Mean',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Min',
  'Pkt Len Std',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'Subflow Fwd Byts',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 4: ['ACK Flag Cnt',
  'Active Max',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd IAT Min',
  'Bwd IAT Tot',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'FIN Flag Cnt',
  'Flow Byts/s',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Fwd Header Len',
  'Fwd IAT Mean',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkts/s',
  'Idle Max',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Min',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'Subflow Fwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 5: ['ACK Flag Cnt',
  'Active Max',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Min',
  'Bwd IAT Tot',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Flow Byts/s',
  'Flow IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Idle Max',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 6: ['ACK Flag Cnt',
  'Active Max',
  'Fwd Pkt Len Mean',
  'Idle Max',
  'Idle Min',
  'Pkt Len Max',
  'Pkt Len Var',
  'Protocol',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 7: ['ACK Flag Cnt', 'Active Max', 'Label'],
 8: ['Label']}

In [36]:
say=0
dataset="./results/other/1/"
folder(dataset)
for f in fdict:
    print(f)
    for ii in file_list:
        try:
            say+=1
            train=file_list[ii][0]
            test=file_list[ii][1]
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}_{f}.csv"
            print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}-{f}")
            target_names=target_name(test)
            ML(train,test,output_csv,fdict[f],step,mixed,dataset[2:-1]+"_"+str(f))   
            print("\n\n\n\n")
        except:pass

2
AD-S1-DI-S1-2
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_2    0     1  DT    0.63     0.611   0.57   0.611  0.53   0.596      0    4.142     0.027          0
 1  results/other/1_2    1     2  DT    0.595    0.584   0.53   0.584  0.5    0.557      0    4.444     0.032          0
 2  results/other/1_2    2     3  DT    0.63     0.607   0.578  0.607  0.535  0.595      0    4.349     0.028          0
 3  results/other/1_2    3     4  DT    0.608    0.596   0.542  0.596  0.508  0.57       0    4.415     0.03           0
 4  results/other/1_2    4     5  DT    0.645    0.617   0.58   0.617  0.539  0.612      0    4.385     0.031          0
 5  results/other/1_2    5     6  DT    0.606    0.601   0.564  0.601  0.521  0.569      0    4.388     0.031          0
 6  results/othe

AD-S1-DI-S2-3
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_3    0     1  DT    0.728    0.734   0.744  0.734  0.706  0.713      0    1.181     0.031          0
 1  results/other/1_3    1     2  DT    0.757    0.751   0.755  0.751  0.725  0.744      0    1.156     0.025          0
 2  results/other/1_3    2     3  DT    0.73     0.737   0.752  0.737  0.707  0.715      0    1.101     0.029          0
 3  results/other/1_3    3     4  DT    0.728    0.731   0.748  0.731  0.703  0.713      0    1.08      0.024          0
 4  results/other/1_3    4     5  DT    0.741    0.736   0.747  0.736  0.715  0.727      0    1.097     0.024          0
 5  results/other/1_3    5     6  DT    0.725    0.726   0.749  0.726  0.702  0.71       0    1.098     0.027          0
 6  results/other/

DI-S2-AD-S1-3
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_3    0     1  DT    0.67     0.686   0.691  0.686  0.652  0.652      0    1.161     0.024          0
 1  results/other/1_3    1     2  DT    0.674    0.692   0.713  0.692  0.667  0.656      0    1.133     0.023          0
 2  results/other/1_3    2     3  DT    0.673    0.691   0.711  0.691  0.656  0.655      0    1.126     0.024          0
 3  results/other/1_3    3     4  DT    0.671    0.687   0.712  0.687  0.648  0.653      0    1.191     0.024          0
 4  results/other/1_3    4     5  DT    0.675    0.693   0.739  0.693  0.664  0.657      0    1.13      0.027          0
 5  results/other/1_3    5     6  DT    0.668    0.684   0.707  0.684  0.659  0.649      0    1.166     0.027          0
 6  results/other/

AD-S2-DI-S2-3
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_3    0     1  DT    0.694    0.692   0.734  0.692  0.654  0.677      0    0.854     0.021          0
 1  results/other/1_3    1     2  DT    0.684    0.684   0.735  0.684  0.649  0.667      0    0.877     0.024          0
 2  results/other/1_3    2     3  DT    0.694    0.691   0.723  0.691  0.653  0.678      0    0.845     0.024          0
 3  results/other/1_3    3     4  DT    0.701    0.698   0.747  0.698  0.661  0.685      0    0.948     0.026          0
 4  results/other/1_3    4     5  DT    0.7      0.697   0.731  0.697  0.659  0.683      0    0.852     0.028          0
 5  results/other/1_3    5     6  DT    0.691    0.689   0.741  0.689  0.653  0.674      0    0.858     0.022          0
 6  results/other/

DI-S2-AD-S2-3
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_3    0     1  DT    0.698    0.715   0.715  0.715  0.69   0.682      0    1.072     0.026          0
 1  results/other/1_3    1     2  DT    0.707    0.724   0.738  0.724  0.693  0.691      0    1.056     0.022          0
 2  results/other/1_3    2     3  DT    0.706    0.724   0.736  0.724  0.699  0.69       0    1.065     0.024          0
 3  results/other/1_3    3     4  DT    0.709    0.726   0.735  0.726  0.707  0.694      0    1.044     0.021          0
 4  results/other/1_3    4     5  DT    0.698    0.714   0.702  0.714  0.683  0.681      0    1.128     0.024          0
 5  results/other/1_3    5     6  DT    0.701    0.718   0.722  0.718  0.677  0.685      0    1.042     0.021          0
 6  results/other/

...

DI-S2-AD-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.47     0.511   0.706  0.511  0.512  0.446      0    0.133     0.015          0
 1  results/other/1_7    1     2  DT    0.471    0.512   0.708  0.512  0.513  0.447      0    0.123     0.016          0
 2  results/other/1_7    2     3  DT    0.476    0.518   0.745  0.518  0.527  0.453      0    0.129     0.016          0
 3  results/other/1_7    3     4  DT    0.472    0.513   0.699  0.513  0.514  0.448      0    0.119     0.016          0
 4  results/other/1_7    4     5  DT    0.472    0.513   0.699  0.513  0.514  0.448      0    0.121     0.016          0
 5  results/other/1_7    5     6  DT    0.472    0.512   0.706  0.512  0.514  0.448      0    0.121     0.017          0
 6  results/other/

AD-S2-DI-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.495    0.51    0.74   0.51   0.537  0.465      0    0.081     0.015          0
 1  results/other/1_7    1     2  DT    0.495    0.51    0.741  0.51   0.537  0.465      0    0.082     0.017          0
 2  results/other/1_7    2     3  DT    0.495    0.51    0.74   0.51   0.537  0.465      0    0.08      0.014          0
 3  results/other/1_7    3     4  DT    0.495    0.51    0.741  0.51   0.537  0.465      0    0.083     0.016          0
 4  results/other/1_7    4     5  DT    0.496    0.511   0.742  0.511  0.538  0.467      0    0.081     0.017          0
 5  results/other/1_7    5     6  DT    0.496    0.511   0.742  0.511  0.538  0.466      0    0.074     0.016          0
 6  results/other/

DI-S2-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.478    0.515   0.778  0.515  0.548  0.456      0    0.128     0.015          0
 1  results/other/1_7    1     2  DT    0.474    0.511   0.725  0.511  0.534  0.451      0    0.126     0.015          0
 2  results/other/1_7    2     3  DT    0.479    0.516   0.778  0.516  0.547  0.456      0    0.111     0.013          0
 3  results/other/1_7    3     4  DT    0.48     0.518   0.779  0.518  0.551  0.458      0    0.115     0.015          0
 4  results/other/1_7    4     5  DT    0.48     0.517   0.779  0.517  0.55   0.457      0    0.124     0.014          0
 5  results/other/1_7    5     6  DT    0.479    0.516   0.778  0.516  0.548  0.456      0    0.117     0.015          0
 6  results/other/

AD-S2-DI-S1-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.496    0.519   0.764  0.519  0.54   0.467      0    0.077     0.015          0
 1  results/other/1_7    1     2  DT    0.495    0.518   0.762  0.518  0.539  0.466      0    0.076     0.016          0
 2  results/other/1_7    2     3  DT    0.494    0.517   0.764  0.517  0.538  0.465      0    0.087     0.016          0
 3  results/other/1_7    3     4  DT    0.495    0.518   0.764  0.518  0.539  0.466      0    0.075     0.015          0
 4  results/other/1_7    4     5  DT    0.495    0.518   0.762  0.518  0.539  0.466      0    0.074     0.015          0
 5  results/other/1_7    5     6  DT    0.496    0.519   0.766  0.519  0.54   0.467      0    0.08      0.015          0
 6  results/other/

DI-S1-AD-S2-7
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/1_7    0     1  DT    0.481    0.512   0.721  0.512  0.521  0.457      0    0.106     0.015          0
 1  results/other/1_7    1     2  DT    0.481    0.512   0.719  0.512  0.521  0.457      0    0.109     0.014          0
 2  results/other/1_7    2     3  DT    0.479    0.509   0.719  0.509  0.517  0.455      0    0.111     0.014          0
 3  results/other/1_7    3     4  DT    0.479    0.509   0.72   0.509  0.517  0.455      0    0.113     0.015          0
 4  results/other/1_7    4     5  DT    0.478    0.509   0.718  0.509  0.517  0.454      0    0.109     0.015          0
 5  results/other/1_7    5     6  DT    0.479    0.509   0.718  0.509  0.517  0.455      0    0.113     0.014          0
 6  results/other/

In [37]:
f

8

In [38]:
features=[['ACK Flag Cnt',
  'Active Max',
  'Active Min',
  'Active Std',
  'Bwd IAT Min',
  'Bwd IAT Std',
  'Bwd IAT Tot',
  'Bwd Pkts/s',
  'Bwd Seg Size Avg',
  'FIN Flag Cnt',
  'Flow IAT Max',
  'Flow IAT Std',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Min',
  'Idle Max',
  'Idle Min',
  'Idle Std',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Min',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Bwd IAT Max',
  'Bwd IAT Mean',
  'Bwd IAT Min',
  'Bwd IAT Std',
  'Bwd IAT Tot',
  'Bwd Pkt Len Min',
  'FIN Flag Cnt',
  'Flow Byts/s',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow Pkts/s',
  'Fwd Act Data Pkts',
  'Fwd IAT Min',
  'Fwd IAT Tot',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Min',
  'Idle Mean',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Min',
  'Pkt Len Var',
  'Subflow Bwd Byts',
  'Subflow Bwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Active Max',
  'Active Min',
  'Bwd IAT Mean',
  'Bwd IAT Min',
  'Bwd IAT Std',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Dst Port',
  'FIN Flag Cnt',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Min',
  'Fwd Header Len',
  'Fwd IAT Max',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Fwd Pkts/s',
  'Fwd Seg Size Avg',
  'Idle Max',
  'Pkt Len Std',
  'Pkt Size Avg',
  'Protocol',
  'Subflow Fwd Pkts',
  'Tot Bwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Active Max',
  'Active Mean',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Min',
  'Bwd IAT Tot',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'FIN Flag Cnt',
  'Flow Byts/s',
  'Flow Duration',
  'Flow IAT Max',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Std',
  'Fwd Pkts/s',
  'Idle Max',
  'Idle Mean',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Min',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'Tot Fwd Pkts',
  'TotLen Bwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Active Max',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd IAT Mean',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Min',
  'Flow Byts/s',
  'Flow Duration',
  'Flow IAT Min',
  'Fwd IAT Mean',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkts/s',
  'Idle Max',
  'Idle Min',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Std',
  'Pkt Len Var',
  'Src Port',
  'Tot Bwd Pkts',
  'TotLen Bwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Active Max',
  'Active Mean',
  'Active Min',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd IAT Tot',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'Bwd Pkts/s',
  'Bwd Seg Size Avg',
  'Flow Byts/s',
  'Flow IAT Max',
  'Flow IAT Min',
  'Flow IAT Std',
  'Fwd Header Len',
  'Fwd IAT Mean',
  'Fwd IAT Min',
  'Fwd IAT Std',
  'Fwd IAT Tot',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Min',
  'Idle Max',
  'Idle Mean',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Min',
  'Pkt Len Std',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'Subflow Fwd Byts',
  'Subflow Fwd Pkts',
  'Tot Fwd Pkts',
  'TotLen Fwd Pkts',
  'Label'],
 ['ACK Flag Cnt',
  'Active Max',
  'Active Mean',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Min',
  'Bwd IAT Tot',
  'Bwd Pkt Len Mean',
  'Bwd Pkts/s',
  'Flow IAT Std',
  'Flow Pkts/s',
  'Fwd Act Data Pkts',
  'Fwd Header Len',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Idle Max',
  'Idle Min',
  'Pkt Len Mean',
  'Protocol',
  'Src Port',
  'Subflow Bwd Byts',
  'TotLen Fwd Pkts',
  'Label'],
 ['Active Max',
  'Active Std',
  'Bwd Header Len',
  'Bwd IAT Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Dst Port',
  'Flow Byts/s',
  'Flow Duration',
  'Flow IAT Mean',
  'Fwd IAT Max',
  'Fwd IAT Mean',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Fwd Pkts/s',
  'Fwd Seg Size Avg',
  'Idle Min',
  'Idle Std',
  'Init Bwd Win Byts',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Var',
  'Pkt Size Avg',
  'Protocol',
  'Subflow Bwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Fwd Pkts',
  'TotLen Bwd Pkts',
  'Label']]

In [39]:
len(features)

8

In [40]:
say=0
dataset="./results/other/2/"
folder(dataset)
for ff,f in enumerate( features):
    print(ff)
    for ii in file_list:
            say+=1
            train=file_list[ii][0]
            test=file_list[ii][1]
            mixed=False
            step=1
            sayac=1
            output_csv=f"{dataset}/@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}_{ff}.csv"
            print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}-{ff}")
            target_names=target_name(test)
            ML(train,test,output_csv,f,step,mixed,dataset[2:-1]+"_"+str(ff))   
            print("\n\n\n\n")

0
AD-S1-DI-S1-0
    Dataset              T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  results/other/2_0    0     1  DT    0.522    0.538   0.534  0.538  0.494  0.477      0    2.152     0.023          0
 1  results/other/2_0    1     2  DT    0.519    0.543   0.539  0.543  0.487  0.474      0    2.11      0.015          0
 2  results/other/2_0    2     3  DT    0.537    0.556   0.549  0.556  0.494  0.494      0    2.15      0.017          0
 3  results/other/2_0    3     4  DT    0.533    0.545   0.537  0.545  0.495  0.489      0    2.061     0.031          0
 4  results/other/2_0    4     5  DT    0.531    0.556   0.544  0.556  0.495  0.489      0    2.04      0.016          0
 5  results/other/2_0    5     6  DT    0.528    0.527   0.541  0.527  0.481  0.487      0    2.019     0.032          0
 6  results/othe